In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/src/sensorcnn")

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import pandas as pd
import matplotlib
matplotlib.style.use('ggplot')

In [3]:
from visualization.examples import *
from dataset.utils import *

In [4]:
all_samples = {}
for filename in csv_file_iterator("/data/Setup Data/Mo"):
    samples = load_from_csv(filename)
    all_samples[filename] = samples

In [5]:
sample1 = all_samples['/data/Setup Data/Mo/2016-05-24T22-36-02Z.csv']

In [6]:
from functools import reduce
df = reduce(lambda x, y: x.append(y), all_samples.values())

In [7]:
df['label'].value_counts()

no_exercise                                                   28987
resistanceTargeted:shoulders/dumbbell-shoulder-press           6859
resistanceTargeted:arms/triceps-extension                      6673
resistanceTargeted:shoulders/lateral-raise                     5346
resistanceTargeted:arms/dumbbell-biceps-curl                   5118
setup_resistanceTargeted:arms/triceps-extension                2750
setup_resistanceTargeted:arms/dumbbell-biceps-curl             2500
setup_resistanceTargeted:shoulders/dumbbell-shoulder-press     2500
setup_resistanceTargeted:shoulders/lateral-raise               2250
Name: label, dtype: int64

In [8]:
labels = [l for l in df['label'].value_counts().keys() if l.startswith("setup_")]
labels_map = {l:i+1 for (i,l) in enumerate(labels)}
label_to_idx = lambda l: labels_map.get(l, 0)

In [9]:
df['target'] = df['label'].map(label_to_idx)

In [10]:
samples = df[['x', 'y', 'z']].as_matrix()

In [11]:
positive_targets = df[df['target'] > 0]
# negative_targets = df[df['target'] == 0]
# targets = np.append(positive_targets[['target']].as_matrix(),
#                     negative_targets[['target']].as_matrix(),
#                     axis=0)

In [12]:
# User positive only
targets = positive_targets[['target']].as_matrix()
samples = df[['x', 'y', 'z']].as_matrix()[positive_targets.index]

In [13]:
df['label'].map(label_to_idx).value_counts()

0    52983
1     2750
3     2500
2     2500
4     2250
Name: label, dtype: int64

In [14]:
print(samples.shape)
print(targets.shape)

(10000, 3)
(10000, 1)


In [15]:
X, y = augment_examples(samples, targets, new_sample_size=50*5)

In [16]:
print(X.shape)
print(y.shape)

(40, 750)
(40, 1)


### Examples encoding

In [17]:
Y = np.zeros((y.shape[0], 4))
iy = np.array([[i, yi-1] for (i, yi) in enumerate(y) if yi > 0])
Y[iy[:, 0], iy[:, 1]] = 1

### Split dataset

In [18]:
X_train, X_test, Y_train, Y_test = split(X, Y)

In [19]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
model.add(Dense(input_dim=X.shape[1], output_dim=150, activation="tanh", init='uniform'))
model.add(Dropout(0.5))
model.add(Dense(input_dim=150, output_dim=100, activation="tanh", init='uniform'))
model.add(Dropout(0.5))
model.add(Dense(input_dim=100, output_dim=4, activation="tanh", init='uniform'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=["accuracy"])

Using Theano backend.


In [20]:
model.fit(X_train, Y_train, nb_epoch=30, batch_size=128)

Epoch 1/30
28/28 [==============================] - 0s - loss: 0.2498 - acc: 0.1429
Epoch 2/30
28/28 [==============================] - 0s - loss: 0.2305 - acc: 0.4286
Epoch 3/30
28/28 [==============================] - 0s - loss: 0.2066 - acc: 0.3929
Epoch 4/30
28/28 [==============================] - 0s - loss: 0.1905 - acc: 0.5000
Epoch 5/30
28/28 [==============================] - 0s - loss: 0.1833 - acc: 0.4643
Epoch 6/30
28/28 [==============================] - 0s - loss: 0.1726 - acc: 0.4643
Epoch 7/30
28/28 [==============================] - 0s - loss: 0.1670 - acc: 0.6071
Epoch 8/30
28/28 [==============================] - 0s - loss: 0.1655 - acc: 0.4643
Epoch 9/30
28/28 [==============================] - 0s - loss: 0.1721 - acc: 0.5000
Epoch 10/30
28/28 [==============================] - 0s - loss: 0.1646 - acc: 0.5357
Epoch 11/30
28/28 [==============================] - 0s - loss: 0.1552 - acc: 0.5000
Epoch 12/30
28/28 [==============================] - 0s - loss: 0.1532 - a

In [21]:
model.predict(X_test)

array([[ 0.05858711,  0.23962727,  0.39901465,  0.34748599],
       [ 0.52966321,  0.31100461,  0.02855542,  0.18296297],
       [ 0.459472  ,  0.31206068,  0.02152504,  0.2539449 ],
       [-0.10300694,  0.48722726,  0.26156265,  0.27872989],
       [ 0.73739421,  0.05843085, -0.10499434,  0.33498991],
       [ 0.10499161,  0.15567634,  0.42874798,  0.36263362],
       [ 0.50309205,  0.04170291,  0.08505519,  0.44268417],
       [-0.09833112,  0.4687354 ,  0.34268534,  0.19744144],
       [-0.36155024,  0.42100099,  0.15934826,  0.60967731],
       [-0.04527041,  0.61438769,  0.22397788,  0.10068619],
       [ 0.10644583,  0.33987078,  0.10503226,  0.41878414],
       [ 0.03555423,  0.3892473 ,  0.426561  ,  0.15534791]])